In [1]:
import pandas as pd
import os
import glob

In [2]:
#Step1: combine anualized patent data from 2000 to 2021
folder_path = "/Users/liusiyi/Library/CloudStorage/OneDrive-IndianaUniversity/Research/patent/patent data/annualized data/csv data"   #fill the file path here
csv_files = sorted(glob.glob(os.path.join(folder_path, "*.csv")))
df_list = []
for f in csv_files:
    print("Reading:", os.path.basename(f))
    df = pd.read_csv(f, dtype=str)  
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)

combined_df.to_csv("/Users/liusiyi/Desktop/patents_2000_2021_combined_copy.csv", index=False)
print("DONE. Shape:", combined_df.shape)

Reading: 2000.csv
Reading: 2001.csv
Reading: 2002.csv
Reading: 2003.csv
Reading: 2004.csv
Reading: 2005.csv
Reading: 2006.csv
Reading: 2007.csv
Reading: 2008.csv
Reading: 2009.csv
Reading: 2010.csv
Reading: 2011.csv
Reading: 2012.csv
Reading: 2013.csv
Reading: 2014.csv
Reading: 2015.csv
Reading: 2016.csv
Reading: 2017.csv
Reading: 2018.csv
Reading: 2019.csv
Reading: 2020.csv
Reading: 2021.csv
DONE. Shape: (5932265, 60)


In [3]:
keep_cols = ["cpc_sections", "grant_year", "patent_number"] #keep the useful column
combined_df = combined_df[keep_cols]
combined_df.to_csv("/Users/liusiyi/Desktop/patents_2000_2021_minimal.csv", index=False)
print("✅ Saved minimal dataset:", combined_df.shape)

✅ Saved minimal dataset: (5932265, 3)


In [4]:
combined_df = combined_df.rename(columns={'patent_number': 'patent_id'}) #rename patent id for further matching
combined_unique = combined_df.drop_duplicates(subset=['patent_id'], keep='first') #drop duplications and rename the dataset combined_unique
combined_unique.to_csv(
    "/Users/liusiyi/Desktop/patents_2000_2021_unique_copy.csv",
    index=False
)

print("✅ Saved to Desktop! Shape:", combined_unique.shape) #save new file

✅ Saved to Desktop! Shape: (5730702, 3)


In [6]:
#Step 2: Merge annualized data with patents' abstract
tsv_path = "/Users/liusiyi/Library/CloudStorage/OneDrive-IndianaUniversity/Research/patent/patent data/add_vaiable/granted data/g_patent_abstract.tsv"   # enter file path for patent abstract 
df_abs = pd.read_csv(tsv_path, sep="\t", dtype=str, low_memory=False)
merged_df = combined_unique.merge(
    df_abs,
    on='patent_id',
    how='left'  
)
merged_df = merged_df[['patent_id', 'cpc_sections', 'grant_year', 'patent_abstract']] #change column order
output_path = "/Users/liusiyi/Documents/patents_2000_2021_with_abstract_copy.csv"
merged_df.to_csv(output_path, index=False)

nan_count = merged_df['patent_abstract'].isna().sum() #calculate the number of missing abstract (which is 52,2167)
total = len(merged_df) #calculate the number of  abstract (which is 5,730,702)
nan_ratio = nan_count / total #calculate ratio (which is 9.1%)

print("number of missing abstract:", nan_count)
print("Total lines:", total)
print("missing ratio: {:.2%}".format(nan_ratio))

number of missing abstract: 522167
Total lines: 5730702
missing ratio: 9.11%


In [11]:
path2 = "/Users/liusiyi/Library/CloudStorage/OneDrive-IndianaUniversity/Research/patent/1111/12706672/ocpb_assigneeatfiling.csv"  #data from zenodo
df2 = pd.read_csv(path2, dtype=str)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3516106 entries, 0 to 3516105
Data columns (total 20 columns):
 #   Column                            Dtype 
---  ------                            ----- 
 0   patent_id                         object
 1   founding_year                     object
 2   founding_score                    object
 3   VC_backed_assignee                object
 4   VC_score                          object
 5   initassignee_id                   object
 6   initassignee_organization         object
 7   same_as20220630pv                 object
 8   assignee_organization_20220630pv  object
 9   assignee_id_20220630pv            object
 10  initassignee_idxwalk20220630pv    object
 11  Found in OC                       object
 12  company_number                    object
 13  gvkey                             object
 14  CUSIP                             object
 15  first_year_publicly_listed        object
 16  UO_DISCERN                        object
 17  SUB_DISC

In [14]:
path2 = "/Users/liusiyi/Library/CloudStorage/OneDrive-IndianaUniversity/Research/patent/1111/12706672/ocpb_assigneeatfiling.csv"  #data from zenodo
df2 = pd.read_csv(path2, dtype=str)
merged2 = df2.merge(
    merged_df,
    on="patent_id",
    how="left"
) #merge pantent abstact with zenodo data using patent_id as a key
merged2['grant_year'] = pd.to_numeric(merged2['grant_year'], errors='coerce') ## trasnform grant_year to number, or NAN
merged2 = merged2[merged2['grant_year'].between(2000, 2021)] #merge with VC data
merged2.to_csv("/Users/liusiyi/Desktop/vcpatent_2000_2021_copy.csv", index=False) #save final dataset

## Step 3: data cleaning

In [15]:
#Step 3: data cleaning 
df = pd.read_csv("/Users/liusiyi/Desktop/vcpatent_2000_2021_copy.csv") #enter the path for patent data saved in appenddata_code.ipynb
df.info() #check the variable information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544381 entries, 0 to 2544380
Data columns (total 23 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   patent_id                         object 
 1   founding_year                     float64
 2   founding_score                    float64
 3   VC_backed_assignee                float64
 4   VC_score                          float64
 5   initassignee_id                   object 
 6   initassignee_organization         object 
 7   same_as20220630pv                 int64  
 8   assignee_organization_20220630pv  object 
 9   assignee_id_20220630pv            object 
 10  initassignee_idxwalk20220630pv    object 
 11  Found in OC                       float64
 12  company_number                    object 
 13  gvkey                             float64
 14  CUSIP                             object 
 15  first_year_publicly_listed        float64
 16  UO_DISCERN                        fl

In [16]:
dfnew = df[(df['grant_year'] >= 2000) & (df['grant_year'] <= 2021)] #keep patent between 2000 and 2021
dfnew = df[df['cpc_sections'].str.contains('C', na=False)] #keep patent which contains section C
dfnew.shape #check the shape (31,4889,23)

(314889, 23)

In [ ]:
#drop the observations with empty abstracts 
before = len(dfnew)

df_clean = dfnew.dropna(subset=["patent_abstract"])
df_clean = df_clean[df_clean["patent_abstract"].str.strip() != ""]  # move NA value of abstarct

after = len(df_clean)

removed = before - after

print(f"Number of original sample: {before}")
print(f"Number of sample after deleting: {after}")
print(f"Number of deleted: {removed}")

output_path = "/Users/liusiyi/Library/CloudStorage/OneDrive-IndianaUniversity/Research/1202/vc0021_C.csv" #preserved clean version
df_clean.to_csv(output_path, index=False) 